In [28]:
import pandas as pd
import fastparquet
import glob
import multiprocessing as mp
from multiprocessing import Pool
import numpy as np
import time
import random

In [29]:
num_partitions=round(0.95*mp.cpu_count())
all_context_files=[]
from_path='/datanaco/dharp/compounds/datasets/googleV3//'
to_path='/data/dharp/compounds/datasets/contexts/'
for filename in glob.glob(from_path+'*parq'):
    all_context_files.append(filename)
    
len(all_context_files)

100

In [39]:
random.shuffle(all_context_files)
all_context_files_list=np.array_split(all_context_files,2)

In [40]:
len(all_context_files_list)

2

In [42]:
all_context_files_list[0]

array(['/datanaco/dharp/compounds/datasets/googleV3/df_8.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_97.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_9.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_54.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_15.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_35.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_7.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_46.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_76.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_68.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_40.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_36.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_72.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_3.parq',
       '/datanaco/dharp/compounds/datasets/googleV3/df_25.parq',
       '/datanaco/dharp/compo

In [31]:
def context_maker(df):
    df[['w1','w2','w3','w4','w5']]=df.lemma_pos.str.split(" ",expand=True)

    context_df=pd.melt(df,id_vars=['year','count'],value_vars=['w1','w2','w3','w4','w5'])
    
    context_df=context_df.groupby(['value','year'],observed=True)['count'].sum().to_frame()
    context_df.reset_index(inplace=True)
    context_df[['word','pos']]=context_df['value'].str.rsplit('_',expand=True,n=1)
    
    context_df=context_df.groupby(['word','pos','year'],observed=True)['count'].sum().to_frame()
    context_df.reset_index(inplace=True)
    context_df=context_df.loc[context_df.pos.isin(['NOUN','ADJ','VERB','ADV'])]
    context_df['context']=context_df['word']+"_"+context_df['pos']
    context_df=context_df.groupby(['context','year'],observed=True)['count'].sum().to_frame()
    context_df.reset_index(inplace=True)

    return context_df    

In [32]:
def parquet_file_processer(f):
    parquet_context_df_list=[]
    print(f'File num: {f+1} name: {cur_parq_file}')
    cur_parq=fastparquet.ParquetFile(cur_parq_file)
    print(f'Number of groups: {len(cur_parq.row_groups)}')
    for i,df in enumerate(cur_parq.iter_row_groups()):
        print(f'Group {i+1}')
        cur_time=time.time()
        context_df=context_maker(df)
        print(f'Total time taken for Group {i+1}: {round(time.time()-cur_time)} secs')        
        parquet_context_df_list.append(context_df)
    
    parq_context_df=pd.concat(parquet_context_df_list,ignore_index=True)
    print(parq_context_df.shape[0])

    parq_context_df=parq_context_df.groupby(['context','year'])['count'].sum().to_frame()
    print(parq_context_df.shape[0])
    parq_context_df.reset_index(inplace=True)
    return parq_context_df

In [33]:
def parquet_batch_processer(cur_list):
    parquet_batch_df_list=[]
    
    for cur_file in cur_list:
        parquet_batch_df_list.append(parquet_file_processer(cur_file))
        
    batch_context_df=pd.concat(parquet_batch_df_list,ignore_index=True)
    print(batch_context_df.shape[0])
    
    batch_context_df=batch_context_df.groupby(['context','year'])['count'].sum().to_frame()
    batch_context_df.reset_index(inplace=True)
    print(batch_context_df.shape[0])
    
    return batch_context_df

In [45]:
len(all_context_files_list[0])

50

In [ ]:
    pool = Pool(len(cur_parq_file_list))


In [6]:
all_context_df_list=[]
for i,cur_parq_file_list in enumerate(all_context_files_list):
    
    cur_time=time.time()

    print(f'Iteration {i+1}')
    
    pool = Pool(len(cur_parq_file_list))
    print('Started parallelization')
    
    results=pool.map_async(parquet_batch_processer,cur_parq_file_list,chunksize=1)
    pool.close()
    pool.join()
    
    curr_df_list=results.get()
    current_context_df=pd.concat(curr_df_list,ignore_index=True)
    
    print(current_context_df.shape[0])
    
    current_context_df=current_context_df.groupby(['context','year'])['count'].sum().to_frame()
    current_context_df.reset_index(inplace=True)
    print(current_context_df.shape[0])
    
    print(f'Total time taken for Iteration {i+1}: {round(time.time()-cur_time)} secs')        
    all_context_df_list.append(current_context_df)
    
    
final_context_df=pd.concat(all_context_df_list,ignore_index=True)

final_context_df=final_context_df.groupby(['context','year'])['count'].sum().to_frame()
final_context_df.reset_index(inplace=True)
print(final_context_df.shape[0])

#final_context_df.to_pickle(

File num: 1 name: /datanaco/dharp/compounds/datasets/googleV3/df_75.parq
Number of groups: 42


NameError: name 'g' is not defined